![BridgingAI Logo](../bridgingai_logo.png)

# BasicsML - Exercise 3: Linear Discriminants
In this exercise, you will implement (generalized) linear discriminant models to classify some toy data.

Make sure to replace all parts that say
```python
# YOUR CODE HERE
raise NotImplementedError()
```

Happy coding!

# Q1: Least Squares Linear Classifier
We start off with a simple least-squares classifier.
This model learns the weights and bias of a linear discriminant function $y(\mathbf{x})$:

$$y(\mathbf{x}) = \mathbf{w}^T\mathbf{x} + w_0$$

Train a least-squares linear classifier on synthetic 2D training data and evaluate it on the training and test set.
You should have extracted the files containing the training and test data together with this notebook.

We already implemented the dataloading, plotting, and evaluation code. Feel free to take a look at them before continuing with the exercise further below.
You will need to write two functions:
- `leastSquares` trains a linear discriminant using the least-squares objective.
- `linclass` applies a linear classifier on some data and returns the predictions.

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def load_data(name):
    data = np.load(f"{name}.npz")
    return {s: data[s] for s in ("data", "labels")}


def plot_contour(fun):
    # make a regular grid over the whole plot
    x, y = np.linspace(*plt.xlim(), 200), np.linspace(*plt.ylim(), 200)
    xx, yy = np.meshgrid(x, y)
    grid = np.c_[xx.ravel(), yy.ravel()]
    # evaluate function on grid
    res = fun(grid).reshape(*xx.shape)

    # plot contour lines (=decision boundary) and filled contours over whole plot
    plt.contourf(xx, yy, res, colors=["yellow", "blue"], alpha=0.2)
    plt.contour(xx, yy, res, levels=1, colors="k")


def plot_(data, labels, params=None, title=None, basis_fun=None):
    # Plot the data points and the decision line
    plt.subplot()
    if title:
        plt.title(title)

    class1, class2 = data[labels > 0], data[labels < 0]
    plt.scatter(*class1.T, c="blue", marker="x")
    plt.scatter(*class2.T, c="orange", marker="o")

    if params:
        w, b = params
        xmax = data[:, 0].max(0)
        xmin = data[:, 0].min(0)
        # Quick and hacky way to fix the y-axis limits
        plt.ylim(plt.ylim())

        if basis_fun:
            # evaluate decision function on grid and plot contours
            plot_contour(lambda grid: linclass(w, b, basis_fun(grid)))
        else:
            # just plot a line
            y = lambda x: -(w[0] * x + b) / w[1]
            plt.plot([xmin, xmax], [y(xmin), y(xmax)], c="k")

    plt.show()

## Q1.1
Implement the `leastSquares` function.
It should train a least-squares classifier based on the data matrix $\mathbf{X}$ and its class label vector $\mathbf{t}$.
As output, it produces the linear classifier weight vector $\mathbf{w}$ and bias $b$.

In [ ]:
def leastSquares(data, label):
    # Minimize the sum-of-squares error
    #
    # INPUT:
    # data        : Training inputs  (num_samples x dim)
    # label       : Training targets- 1D array of length {num_samples}
    #
    # OUTPUT:
    # weights     : weights- 1D array of length {dim}
    # bias        : bias term (scalar)

    # YOUR CODE HERE
    raise NotImplementedError()

    return weight, bias

## Q1.2
Implement the function `linclass` that classifies a data matrix $\mathbf{X}$ based on a trained linear classifier given by $\mathbf{w}$ and $b$.
Remember that we expect classification results to be either 1 or -1.

In [ ]:
def linclass(weight, bias, data):
    # Apply a linear classifier
    #
    # INPUT:
    # weight      : weights                1D array of length {dim}
    # bias        : bias term              (scalar)
    # data        : Input to be classified (num_samples x dim)
    #
    # OUTPUT:
    # class_pred       : Predicted class (+-1) values- 1D array of length {num_samples}

    # YOUR CODE HERE
    raise NotImplementedError()

    return class_pred

## Q1.3
Run the cells below to train and evaluate a linear classifier on the provided data.
Analyze the classification plots for both the datasets. Are the sets optimally classified?

In [ ]:
train = load_data("lc_train")
test = load_data("lc_test")


def evaluate(data, label, params, title, basis_fun=None):
    pred = linclass(weight, bias, basis_fun(data) if basis_fun is not None else data)
    acc = (pred == label).mean()
    print(f"Accuracy on {title}: {acc:.5f}")
    plot_(data, label, params, title, basis_fun)

In [ ]:
weight, bias = leastSquares(train["data"], train["labels"])

# Evaluate on the train set
evaluate(train["data"], train["labels"], (weight, bias), "Train Set")

# Evaluate on the test set
evaluate(test["data"], test["labels"], (weight, bias), "Test Set")

## Q1.4
Now we add some outliers to the data.
Run the cells below to train and evaluate the classifier on the augmented data.

Again, analyze the classification plots for both datasets. What do you notice?

In [ ]:
outlier_train = {
    "data": np.append(train["data"], [[1.5, -0.4], [1.45, -0.35]], axis=0),
    "labels": np.append(train["labels"], [[-1], [-1]]),
}

weight, bias = leastSquares(outlier_train["data"], outlier_train["labels"])

# Evaluate on the train set
evaluate(
    outlier_train["data"],
    outlier_train["labels"],
    (weight, bias),
    "Train Set with Outliers",
)

# Evaluate on the test set
evaluate(test["data"], test["labels"], (weight, bias), "Test Set")

# Q2: Basis Functions
Now we will implement polynomial basis functions and use them to classifiy non-linearly separable data.
Our basis functions will have the following form:

To start, implement $\phi_2(\mathbf{x})$, that is, a second degree polynomial basis function, for two-dimensional data:

$$
\phi_2(\mathbf{x}) = (1, x_1, x_2, x_1^2, x_1 x_2, x_2^2)^T
$$

In [ ]:
def poly_two(x):
    # YOUR CODE HERE
    raise NotImplementedError()


w = np.array([0.25, -0.5, -0.5, 0, 0, 1])
plot_contour(lambda x: linclass(w, 0, poly_two(x)))
plt.show()

Now train a linear discriminant using this basis function.

In [ ]:
train_poly = poly_two(outlier_train["data"])
test_poly = poly_two(test["data"])

weight, bias = leastSquares(train_poly, outlier_train["labels"])

# Evaluate on the train set
evaluate(
    outlier_train["data"],
    outlier_train["labels"],
    (weight, bias),
    "Train Set",
    poly_two,
)

# Evaluate on the test set
evaluate(test["data"], test["labels"], (weight, bias), "Test Set", poly_two)

If you want to play around, you may also implement a general polynomial basis function for arbitrary degrees.
What do you notice when increasing the degree of the polynomial?
What effect do outliers have on the classification result and the decision boundary?

# Q3: Generalized Linear Discriminants and Iterative Optimization
In order to limit the influence of outliers, we will now add activation functions to our discriminants:
    
$$
y(\mathbf{x}) = \sigma(\mathbf{w}^T \phi(\mathbf{x}) + b)
$$

Since there is in general no closed-form solution to the training objective any more, we will use gradient descent to optimize it iteratively.

In order to do this, you need to implement the following functions:
- the activation function itself (we will use the tangens hyperbolicus discussed in the lecture)
- the derivative of the activation function
- the gradient descent update equation

The rest is provided by us.

## Q3.1: Implement Tanh
Implement the tanh function as

$$
\text{tanh}(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
$$

Show that the derivative of $\text{tanh}$ is

$$
\text{tanh}^\prime(x) = 1 - \text{tanh}(x)^2
$$

Implement the `tanh` and `grad_tanh` functions below; they should apply $\text{tanh}$ (and $\text{tanh}^\prime$, respectively) element-wise to the input.
Check the resulting plot to see whether your implementation makes sense.

In [ ]:
def tanh(x):
    # YOUR CODE HERE
    raise NotImplementedError()


def grad_tanh(x):
    # YOUR CODE HERE
    raise NotImplementedError()


def plot__(*fs):
    x = np.linspace(-5, 5, 400)
    for f in fs:
        plt.plot(x, f(x), label=f.__name__)
    plt.legend()
    plt.show()


plot__(tanh)

Below, we implemented a method to check your analytical derivative against a numerical approximation, using

$$
f^\prime(x) = \lim_{\epsilon\rightarrow0} \frac{f(x+\epsilon) - f(x-\epsilon)}{2\epsilon}
$$

You can run it to test whether your derivative matches your activation function.

In [ ]:
def check_grad(f, f_grad, eps=1e-5):
    x = np.random.randn(100)
    num_grad = (f(x + eps) - f(x - eps)) / (2 * eps)
    if np.allclose(num_grad, f_grad(x)):
        print(f"Grad for {f.__name__} seems correct")
    else:
        print(f"Numeric and analytical grad for {f.__name__} differ:")
        print(num_grad - f_grad(x))


check_grad(tanh, grad_tanh)

## Q3.2: Implement Gradient Descent
Implement the missing parts of the gradient descent algorithm below.
Run the code to train a model on the data that we provided.

To do this, you should first calculate the error gradient with respect to the learnable parameters, then use the gradient to update these parameters.
Remember the formula for gradient descent from the lecture:

$$
w^{(\tau+1)} = w^{(\tau)} - \eta \nabla E(w^{(\tau)}),
$$

using the squared error function and a generalized linear discriminant:

$$
E = \frac{1}{2} \sum_{n=1}^N (\sigma(w^T \phi(x_n)) - t_n)^2
$$

$$
\nabla E = \sum_{n=1}^N (y_n - t_n) \sigma^{\prime}(w^T \phi(x_n)) \phi(x_n).
$$

Use the `tanh` and `grad_tanh` functions that you implemented above.


What results do you get?

In [ ]:
def grad_descent(eta, iters, data, label):
    N = len(data)
    data = np.concatenate((np.ones((N, 1)), data), axis=1)
    w = np.random.randn(data.shape[1])

    errors = []
    for i in range(iters):
        # YOUR CODE HERE
        raise NotImplementedError()

    plt.plot(errors)
    plt.title("Error over gradient descent iterations")
    plt.ylabel("Error")
    plt.xlabel("Iteration")
    plt.show()

    # first entry is bias term
    return w[1:], w[0]

eta = 0.01
iters = 1000

weight, bias = grad_descent(eta, iters, outlier_train["data"], outlier_train["labels"])

# Evaluate on the train set
evaluate(outlier_train["data"], outlier_train["labels"], (weight, bias), "Train Set")

# Evaluate on the test set
evaluate(test["data"], test["labels"], (weight, bias), "Test Set")

Let's try this on some different datasets.

In [ ]:
# try different dataset
train_XOR = load_data("XOR_train")
test_XOR = load_data("XOR_test")

weight, bias = grad_descent(eta, iters, train_XOR["data"], train_XOR["labels"])

# Evaluate on the train set
evaluate(train_XOR["data"], train_XOR["labels"], (weight, bias), "Train Set")

# Evaluate on the test set
evaluate(test_XOR["data"], test_XOR["labels"], (weight, bias), "Test Set")

The previous outcome is unsatisfactory due to the XOR dataset's lack of linear separability. Therefore, we can attempt to incorporate a kernel function to allow our classifier to establish a nonlinear decision boundary.

In [ ]:
"""try poly_two kernel"""

# try different dataset
train_XOR = load_data("XOR_train")
test_XOR = load_data("XOR_test")

weight, bias = grad_descent(eta, iters, poly_two(train_XOR["data"]), train_XOR["labels"])

# Evaluate on the train set
evaluate(train_XOR["data"], train_XOR["labels"], (weight, bias), "Train Set", poly_two)

# Evaluate on the test set
evaluate(test_XOR["data"], test_XOR["labels"], (weight, bias), "Test Set", poly_two)

In [ ]:
# try different dataset
train_concentric = load_data("Concentric_train")
test_concentric = load_data("Concentric_test")

weight, bias = grad_descent(eta, iters, poly_two(train_concentric["data"]), train_concentric["labels"])

# Evaluate on the train set
evaluate(train_concentric["data"], train_concentric["labels"], (weight, bias), "Train Set", poly_two)

# Evaluate on the test set
evaluate(test_concentric["data"], test_concentric["labels"], (weight, bias), "Test Set", poly_two)

In [ ]:
# try different dataset
train_CC = load_data("CC_train")
test_CC = load_data("CC_test")

weight, bias = grad_descent(eta, iters, poly_two(train_CC["data"]), train_CC["labels"])

# Evaluate on the train set
evaluate(train_CC["data"], train_CC["labels"], (weight, bias), "Train Set", poly_two)

# Evaluate on the test set
evaluate(test_CC["data"], test_CC["labels"], (weight, bias), "Test Set", poly_two)

## Q3.3: Enhancing the polynomial kernel
The previous outcome does not meet our expectations as we have only utilized a 2nd-degree polynomial kernel. To improve the results, let's consider using a higher-order polynomial kernel!

Please complete the missing section in the poly function below. Afterward, execute the code to train a model using the provided data.

In [ ]:
def poly(x):
    # YOUR CODE HERE
    raise NotImplementedError()
    
train_CC = load_data("CC_train")
test_CC = load_data("CC_test")

eta = 0.01
iters = 10000
weight, bias = grad_descent(eta, iters, poly(train_CC["data"]), train_CC["labels"])

# Evaluate on the train set
evaluate(train_CC["data"], train_CC["labels"], (weight, bias), "Train Set", poly)

# Evaluate on the test set
evaluate(test_CC["data"], test_CC["labels"], (weight, bias), "Test Set", poly)